# 뉴럴 네트워크 학습 알고리즘 구현

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

## 데이터 불러오기

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# set mini-batch
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 네트워크 구조 정의

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.out_put = tf.keras.layers.Dense(10, activation='softmax')       
    
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.out_put(x)

## 학습 함수 구현

In [5]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    
    with tf.GradientTape() as tape:
        predictions = model(images)  # predictions = BatchSize x 10 (Classes)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [6]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)  # BatchSize x 10(Classes)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 모델 생성

In [7]:
model = MyModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # y : 0 ~ 9
# loss_object = tf.keras.losses.CategoricalCrossentropy()     # y : one-hot encoding

optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [8]:
# batch로 진행되기 때문에 누적된 loss를 평균함
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [9]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {:.5f}, Accuracy: {:.3f}, Test Loss: {:.5f}, Test Accuracy: {:.3f}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.28871, Accuracy: 91.153, Test Loss: 0.17597, Test Accuracy: 94.740
Epoch 2, Loss: 0.14583, Accuracy: 95.493, Test Loss: 0.15685, Test Accuracy: 95.470
Epoch 3, Loss: 0.11360, Accuracy: 96.473, Test Loss: 0.15736, Test Accuracy: 95.490
Epoch 4, Loss: 0.09628, Accuracy: 96.993, Test Loss: 0.13776, Test Accuracy: 95.850
Epoch 5, Loss: 0.08181, Accuracy: 97.413, Test Loss: 0.12325, Test Accuracy: 96.530
Epoch 6, Loss: 0.07344, Accuracy: 97.718, Test Loss: 0.12334, Test Accuracy: 96.920
Epoc